In [58]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [59]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nhl_pop = cities.set_index('Metropolitan area')[['Population (2016 est.)[8]','NHL']]
nhl_pop['NHL'] = nhl_pop['NHL'].replace("—", "").str.replace("\[.*\]","",regex=True)
nhl_pop = nhl_pop[nhl_pop['NHL'].str.contains('\w+')]

In [60]:
nhl_df=pd.read_csv("assets/nhl.csv")
nhl = nhl_df[nhl_df['year'] == 2018]
nhl = nhl.drop(nhl.index[nhl['team'].str.contains('Division')])
nhl['team'] = nhl['team'].str.replace('*', '')

In [61]:

def split_team_names(string):
    split = string.split(' ')
    if len(split) > 1:
        new_split = []
        while split != []:
            if split[0][-1] == 's': 
                new_split.append(split.pop(0))
                if split == []: break
            if split[0][-1] != 's': 
                if len(split) == 1: new_split.append(split.pop(0))
                else: new_split.append(' '.join([split.pop(0), split.pop(0)]))
        return new_split
    else: return split

def match_teams(region_team_string, team_to_match):
    for region_team in split_team_names(region_team_string): 
        if region_team in team_to_match: return region_team

def match_metro(row): return nhl_pop['NHL'].apply(lambda x: match_teams(x, row)).dropna().index[0]
nhl['Metropolitan area'] = nhl['team'].apply(match_metro)
nhl['W/L ratio'] = nhl['W'].apply(int) / (nhl['W'].apply(int) + nhl['L'].apply(int))
city_ratios = nhl[['W/L ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
nhl_pop = nhl_pop.merge(city_ratios, on='Metropolitan area')


In [ ]:

def nhl_correlation(): 
    population_by_region = nhl_pop['Population (2016 est.)[8]'].apply(int) # pass in metropolitan area population from cities
    win_loss_by_region = nhl_pop['W/L ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    nba = nba_df[lambda x: x['year'] == 2018]
    nba['team'] = nba['team'].str.replace('*', '').str.replace("\(.*\)","")
    nba['W/L ratio'] = nba['W'].apply(int) / (nba['W'].apply(int) + nba['L'].apply(int))
    
    all_pop = cities.set_index('Metropolitan area')[['Population (2016 est.)[8]','NBA']]
    all_pop['NBA'] = all_pop['NBA'].replace("—", "").str.replace("\[.*\]","")
    global nba_pop
    nba_pop = all_pop[all_pop['NBA'].str.contains('\w+')]
    
    def match_metro(row): return nba_pop['NBA'].apply(lambda x: match_teams(x, row)).dropna().index[0]
    nba['Metropolitan area'] = nba['team'].apply(match_metro)
    city_ratios = nba[['W/L ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    nba_pop = nba_pop.merge(city_ratios, on='Metropolitan area')
    
    population_by_region = nba_pop['Population (2016 est.)[8]'].apply(int) # pass in metropolitan area population from cities
    win_loss_by_region = nba_pop['W/L ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


nba_correlation()

(-0.17657160252844617, 0.36874741604463)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    mlb = mlb_df[lambda x: x['year'] == 2018]
    mlb['W/L ratio'] = mlb['W-L%']
        
    all_pop = cities.set_index('Metropolitan area')[['Population (2016 est.)[8]','MLB']]
    all_pop['MLB'] = all_pop['MLB'].replace("—", "").str.replace("\[.*\]","")
    global mlb_pop
    mlb_pop = all_pop[all_pop['MLB'].str.contains('\w+')]
    
    def match_metro(row): return mlb_pop['MLB'].apply(lambda x: match_teams(x, row)).dropna().index[0]
    mlb['Metropolitan area'] = mlb['team'].apply(match_metro)
    city_ratios = mlb[['W/L ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    mlb_pop = mlb_pop.merge(city_ratios, on='Metropolitan area')
    
    population_by_region = mlb_pop['Population (2016 est.)[8]'].apply(int) # pass in metropolitan area population from cities
    win_loss_by_region = mlb_pop['W/L ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

(0.1500373747540949, 0.46442827201123393)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    nfl = nfl_df[lambda x: x['year'] == 2018]
    nfl = nfl.drop(nfl.index[nfl['team'].str.contains('FC')])
    nfl['team'] = nfl['team'].str.replace('[*+]', '')
    nfl['W/L ratio'] = nfl['W'].apply(int) / (nfl['W'].apply(int) + nfl['L'].apply(int))
    
    all_pop = cities.set_index('Metropolitan area')[['Population (2016 est.)[8]','NFL']]
    all_pop['NFL'] = all_pop['NFL'].str.replace("\[.*\]","")
    global nfl_pop
    nfl_pop = all_pop[all_pop['NFL'].str.contains('\w+')]
        
    def match_metro(row): return nfl_pop['NFL'].apply(lambda x: match_teams(x, row)).dropna().index[0]
    nfl['Metropolitan area'] = nfl['team'].apply(match_metro)
    city_ratios = nfl[['W/L ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    nfl_pop = nfl_pop.merge(city_ratios, on='Metropolitan area')
        
    population_by_region = nfl_pop['Population (2016 est.)[8]'].apply(int) # pass in metropolitan area population from cities
    win_loss_by_region = nfl_pop['W/L ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

(0.004922112149349414, 0.9797833458363694)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [28]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    pairs = ['NFL_NBA', 'NFL_NHL', 'NFL_MLB', 'NBA_NHL', 'NBA_MLB', 'NHL_MLB']
        
    NFL_NBA = nfl_pop.merge(nba_pop, on='Metropolitan area')
    NFL_NHL = nfl_pop.merge(nhl_pop, on='Metropolitan area')
    NFL_MLB = nfl_pop.merge(mlb_pop, on='Metropolitan area')
    NBA_NHL = nba_pop.merge(nhl_pop, on='Metropolitan area')
    NBA_MLB = nba_pop.merge(mlb_pop, on='Metropolitan area')
    NHL_MLB = nhl_pop.merge(mlb_pop, on='Metropolitan area')
    
    NFL_NBA_stat = stats.ttest_rel(NFL_NBA['W/L ratio_x'], NFL_NBA['W/L ratio_y'])
    NFL_NHL_stat = stats.ttest_rel(NFL_NHL['W/L ratio_x'], NFL_NHL['W/L ratio_y'])
    NFL_MLB_stat = stats.ttest_rel(NFL_MLB['W/L ratio_x'], NFL_MLB['W/L ratio_y'])
    NBA_NHL_stat = stats.ttest_rel(NBA_NHL['W/L ratio_x'], NBA_NHL['W/L ratio_y'])
    NBA_MLB_stat = stats.ttest_rel(NBA_MLB['W/L ratio_x'], NBA_MLB['W/L ratio_y'])
    NHL_MLB_stat = stats.ttest_rel(NHL_MLB['W/L ratio_x'], NHL_MLB['W/L ratio_y'])
    
    p_values.loc["NFL", "NBA"] = NFL_NBA_stat.pvalue
    p_values.loc["NBA", "NFL"] = NFL_NBA_stat.pvalue
    p_values.loc["NFL", "NHL"] = NFL_NHL_stat.pvalue
    p_values.loc["NHL", "NFL"] = NFL_NHL_stat.pvalue
    p_values.loc["NFL", "MLB"] = NFL_MLB_stat.pvalue
    p_values.loc["MLB", "NFL"] = NFL_MLB_stat.pvalue
    p_values.loc["NBA", "NHL"] = NBA_NHL_stat.pvalue
    p_values.loc["NHL", "NBA"] = NBA_NHL_stat.pvalue
    p_values.loc["NBA", "MLB"] = NBA_MLB_stat.pvalue
    p_values.loc["MLB", "NBA"] = NBA_MLB_stat.pvalue
    p_values.loc["NHL", "MLB"] = NHL_MLB_stat.pvalue
    p_values.loc["MLB", "NHL"] = NHL_MLB_stat.pvalue
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802499
NBA,0.941792,NaN,0.022297,0.951629
NHL,0.030883,0.022297,NaN,0.000703
MLB,0.802499,0.951629,0.000703,NaN
